<a href="https://colab.research.google.com/github/alshahreyaj/thesis/blob/main/ML%2BVGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
training_dir = "/content/drive/MyDrive/Colab Notebooks/Thesis/hemorrhage"
img_dir_list = os.listdir(training_dir)

In [ ]:
'''!pip install pandas
!pip install numpy
!pip install opencv-python
!pip install sklearn'''

'!pip install pandas\n!pip install numpy\n!pip install opencv-python\n!pip install sklearn'

In [ ]:
'''!pip uninstall keras'''

'!pip uninstall keras'

In [ ]:
img_dir_list

['Hemorrhagic', 'Normal']

In [ ]:
class_names = {'Hemorrhagic': 0, 'Normal': 1}
# list file address and class
DATA_DF = []
for img_dir in img_dir_list:
  file_dir = os.listdir(training_dir+"/"+img_dir)
  for file_name in file_dir:
    DATA_DF.append([file_name,class_names[img_dir],img_dir])

In [ ]:
# convert to pandas data frame
import pandas as pd
from sklearn.utils import shuffle

Train_Data_Frame = pd.DataFrame(DATA_DF, columns = ["file_name","class","dir"])

Train_Data_Frame = shuffle(Train_Data_Frame)

Train_Data_Frame.head()

,file_name,class,dir
4990,N25_0_41.jpg,1,Normal
6278,N11_0_68.jpg,1,Normal
5148,N21_0_132.jpg,1,Normal
2341,17_0_108.jpg,0,Hemorrhagic
5360,N21_0_78.jpg,1,Normal


In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
y_data = np.array(to_categorical(Train_Data_Frame["class"]))

In [ ]:
print(y_data[:5])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [ ]:
import time
start = time.time()

import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Model
import tensorflow as tf

In [ ]:

base_model = tf.keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(224, 224, 3))
base_model.summary()
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

80150528/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
import cv2
from tqdm import tqdm
x_data = []
for rows in tqdm(range(len(Train_Data_Frame))):
  get_file_name = Train_Data_Frame.iloc[rows]
  img = cv2.imread(training_dir+'/'+get_file_name['dir']+'/'+get_file_name['file_name'])
  stretch_near = cv2.resize(img, (224, 224),
               interpolation = cv2.INTER_AREA)
  features = model.predict(stretch_near.reshape(1,224,224,3))
  features = features.reshape(25088,)
  x_data.append(features)




100%|██████████| 6795/6795 [25:44<00:00,  4.40it/s]


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x_data,y_data,train_size=0.9,shuffle=True,random_state=42)

### If there is seperate test folder
 

In [ ]:
'''testing_dir = "/content/drive/MyDrive/Colab Notebooks/Image Processing/kidney/Test_images"
img_dir_list = os.listdir(training_dir)
class_names = {'Grade0': 0, 'Grade1': 1, 'Grade3': 2, 'Grade4': 3, 'Grade2': 4}
# list file address and class
DATA_DF = []
for img_dir in img_dir_list:
  file_dir = os.listdir(testing_dir+"/"+img_dir)
  for file_name in file_dir:
    DATA_DF.append([file_name,class_names[img_dir],img_dir])

# convert to pandas data frame
import pandas as pd
from sklearn.utils import shuffle

Test_Data_Frame = pd.DataFrame(DATA_DF, columns = ["file_name","class","dir"])

Test_Data_Frame = shuffle(Test_Data_Frame)

import numpy as np
from keras.utils.np_utils import to_categorical
y_test_data = np.array(to_categorical(Test_Data_Frame["class"]))

'''

'testing_dir = "/content/drive/MyDrive/Colab Notebooks/Image Processing/kidney/Test_images"\nimg_dir_list = os.listdir(training_dir)\nclass_names = {\'Grade0\': 0, \'Grade1\': 1, \'Grade3\': 2, \'Grade4\': 3, \'Grade2\': 4}\n# list file address and class\nDATA_DF = []\nfor img_dir in img_dir_list:\n  file_dir = os.listdir(testing_dir+"/"+img_dir)\n  for file_name in file_dir:\n    DATA_DF.append([file_name,class_names[img_dir],img_dir])\n\n# convert to pandas data frame\nimport pandas as pd\nfrom sklearn.utils import shuffle\n\nTest_Data_Frame = pd.DataFrame(DATA_DF, columns = ["file_name","class","dir"])\n\nTest_Data_Frame = shuffle(Test_Data_Frame)\n\nimport numpy as np\nfrom keras.utils.np_utils import to_categorical\ny_test_data = np.array(to_categorical(Test_Data_Frame["class"]))\n\n'

In [ ]:
'''import cv2
from tqdm import tqdm
x_test_data = []
for rows in tqdm(range(len(Test_Data_Frame))):
  get_file_name = Test_Data_Frame.iloc[rows]
  img = cv2.imread(testing_dir+'/'+get_file_name['dir']+'/'+get_file_name['file_name'])
  stretch_near = cv2.resize(img, (224, 224),
               interpolation = cv2.INTER_AREA)
  features = model.predict(stretch_near.reshape(1,224,224,3))
  features = features.reshape(2048,)
  x_test_data.append(features)'''

"import cv2\nfrom tqdm import tqdm\nx_test_data = []\nfor rows in tqdm(range(len(Test_Data_Frame))):\n  get_file_name = Test_Data_Frame.iloc[rows]\n  img = cv2.imread(testing_dir+'/'+get_file_name['dir']+'/'+get_file_name['file_name'])\n  stretch_near = cv2.resize(img, (224, 224),\n               interpolation = cv2.INTER_AREA)\n  features = model.predict(stretch_near.reshape(1,224,224,3))\n  features = features.reshape(2048,)\n  x_test_data.append(features)"

In [ ]:
'''x_test, y_test = x_test_data, y_test_data'''

'x_test, y_test = x_test_data, y_test_data'

In [ ]:
'''x_train,y_train = x_data, y_data'''

'x_train,y_train = x_data, y_data'

In [ ]:
#from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=42)


#### Grid search

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score

# Initializing Classifiers
clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=1)
clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = DecisionTreeClassifier(random_state=1)
clf4 = SVC(random_state=1)
clf5 = RandomForestClassifier(random_state=1)
clf6 = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
clf7 = GradientBoostingClassifier()
# Building the pipelines
pipe1 = Pipeline([('std', StandardScaler()),
                  ('clf1', clf1)])

pipe2 = Pipeline([('std', StandardScaler()),
                  ('clf2', clf2)])

pipe4 = Pipeline([('std', StandardScaler()),
                  ('clf4', clf4)])



# Setting up the parameter grids
param_grid1 = [{'clf1__penalty': ['l2'],
                'clf1__C': np.power(10., np.arange(-4, 4))}]

param_grid2 = [{'clf2__n_neighbors': list(range(1, 10)),
                'clf2__p': [1, 2]}]

param_grid3 = [{'max_depth': list(range(1, 10)) + [None],
                'criterion': ['gini', 'entropy']}]

param_grid4 = [{'clf4__kernel': ['rbf'],
                'clf4__C': np.power(10., np.arange(-4, 4)),
                'clf4__gamma': np.power(10., np.arange(-5, 0))},
               {'clf4__kernel': ['linear'],
                'clf4__C': np.power(10., np.arange(-4, 4))}]

param_grid5 = [{'n_estimators': [10, 100, 500, 1000]}]

param_grid6 = [{
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }]

param_grid7 = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1]
}


In [ ]:
# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4, param_grid5,param_grid6,param_grid7),
                            (pipe1, pipe2, clf3, pipe4, clf5,clf6,clf7),
                            ('Softmax', 'KNN', 'DTree', 'SVM', 'RForest','Xgboost','GradientBoosting')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=-1,
                       cv=inner_cv,
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv


In [ ]:
'''for name, gs_est in sorted(gridcvs.items()):
        gridcvs[name].fit(np.array(x_train),np.argmax(y_train,axis=1)) # run inner loop hyperparam tuning
        print(name+':')
        print('\n        Best ACC (avg. ) %.2f%%' % (gridcvs[name].best_score_ * 100))
        print('        Best parameters:', gridcvs[name].best_params_)'''

"for name, gs_est in sorted(gridcvs.items()):\n        gridcvs[name].fit(np.array(x_train),np.argmax(y_train,axis=1)) # run inner loop hyperparam tuning\n        print(name+':')\n        print('\n        Best ACC (avg. ) %.2f%%' % (gridcvs[name].best_score_ * 100))\n        print('        Best parameters:', gridcvs[name].best_params_)"

#### Model Training 


In [ ]:
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier

xb = xgb.XGBClassifier()
svm = SVC(kernel='rbf')
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)

classifiers = {"svm": svm, "xboost": xb, "Gradientboosting": gbm}

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std

In [ ]:
np.array(x_train).shape

(6115, 25088)

In [ ]:
np.argmax(y_train, axis=1).shape

(6115,)

In [ ]:
for d in tqdm(classifiers):
  classifier = classifiers[d]
  print(d+" :")
  cv = KFold(n_splits=10, random_state=1, shuffle=True)
  #scores = cross_val_score(classifier, np.array(x_train),np.argmax(y_train,axis=1), scoring='accuracy', cv=cv, n_jobs=-1)
  # report performance
  #print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
  classifier.fit(np.array(x_train),np.argmax(y_train,axis=1))
  training_score = classifier.score(np.array(x_train),np.argmax(y_train,axis=1))
  print("Training Score: "+str(training_score))
  testing_score = classifier.score(np.array(x_test),np.argmax(y_test,axis=1))
  print("Testing Score: "+str(testing_score))
  yhat_classes = classifier.predict(np.array(x_test))
  # precision tp / (tp + fp)
  precision = precision_score(np.argmax(y_test, axis=1),yhat_classes, average='weighted')
  print('Precision: %f' % precision)
  # recall: tp / (tp + fn)
  recall = recall_score(np.argmax(y_test, axis=1),yhat_classes, average='weighted')
  print('Recall: %f' % recall)
  # f1: 2 tp / (2 tp + fp + fn)
  f1 = f1_score(np.argmax(y_test, axis=1),yhat_classes, average='weighted')
  print('F1 score: %f' % f1)
  # kappa
  kappa = cohen_kappa_score(np.argmax(y_test, axis=1),yhat_classes)
  print('Cohens kappa: %f' % kappa)
  # ROC AUC
  #auc = roc_auc_score(np.argmax(y_test, axis=1),yhat_classes,multi_class="ovo", average='weighted')
  #print('ROC AUC: %f' % auc)
  # confusion matrix
  matrix = confusion_matrix(np.argmax(y_test, axis=1),yhat_classes)
  print(matrix)


  0%|          | 0/3 [00:00<?, ?it/s]

svm :
Training Score: 0.9993458708094849
Testing Score: 1.0


 33%|███▎      | 1/3 [20:14<40:29, 1214.82s/it]

Precision: 1.000000
Recall: 1.000000
F1 score: 1.000000
Cohens kappa: 1.000000
[[255   0]
 [  0 425]]
xboost :
Training Score: 0.996238757154538
Testing Score: 0.9911764705882353


 67%|██████▋   | 2/3 [26:08<11:47, 707.98s/it] 

Precision: 0.991181
Recall: 0.991176
F1 score: 0.991169
Cohens kappa: 0.981147
[[251   4]
 [  2 423]]
Gradientboosting :


100%|██████████| 3/3 [37:22<00:00, 747.50s/it]

Training Score: 0.9934587080948487
Testing Score: 0.961764705882353
Precision: 0.961765
Recall: 0.961765
F1 score: 0.961765
Cohens kappa: 0.918431
[[242  13]
 [ 13 412]]
